In [ ]:
import sqlite3
from collections import defaultdict

def connect_database():
    """Kết nối đến database zigbee_packets"""
    try:
        conn = sqlite3.connect('zigbee_packets.db')
        print("✅ Kết nối database thành công")
        return conn
    except Exception as e:
        print(f"❌ Lỗi kết nối database: {e}")
        return None

def get_event_stats_by_id(conn):
    """Thống kê số lượng event theo từng ID"""
    query = """
    SELECT 
        ID,
        event,
        COUNT(*) as count
    FROM zigbee_packets 
    WHERE ID IS NOT NULL AND ID != '' AND event IS NOT NULL AND event != ''
    GROUP BY ID, event
    ORDER BY ID, count DESC
    """
    
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Tổ chức dữ liệu theo ID
    stats = defaultdict(dict)
    for id_val, event, count in results:
        stats[id_val][event] = count
    
    return stats

def display_stats(stats):
    """Hiển thị thống kê"""
    print("\n" + "="*60)
    print("📊 THỐNG KÊ SỐ LƯỢNG EVENT THEO TỪNG ID")
    print("="*60)
    
    for id_val in sorted(stats.keys()):
        print(f"\n🔹 ID: {id_val}")
        print("-" * 40)
        
        events = stats[id_val]
        total_events = sum(events.values())
        
        for event, count in sorted(events.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / total_events) * 100
            print(f"   {event:<25} : {count:>5} events ({percentage:>5.6f}%)")
        
        print(f"   {'TOTAL':<25} : {total_events:>5} events")

def get_summary_table(stats):
    """Tạo bảng tóm tắt"""
    print("\n" + "="*50)
    print("📋 BẢNG TÓM TẮT")
    print("="*50)
    print(f"{'ID':<15} {'Event Types':<12} {'Total Events':<12}")
    print("-" * 50)
    
    grand_total = 0
    for id_val in sorted(stats.keys()):
        events = stats[id_val]
        event_types = len(events)
        total_events = sum(events.values())
        grand_total += total_events
        
        print(f"{id_val:<15} {event_types:<12} {total_events:<12}")
    
    print("-" * 50)
    print(f"{'TOTAL':<15} {'':<12} {grand_total:<12}")

def main():
    """Hàm main"""
    # Kết nối database
    conn = connect_database()
    if not conn:
        return
    
    try:
        # Lấy thống kê
        stats = get_event_stats_by_id(conn)
        
        if not stats:
            print("❌ Không có dữ liệu để thống kê")
            return
        
        # Hiển thị thống kê chi tiết
        display_stats(stats)
        
        # Hiển thị bảng tóm tắt
        get_summary_table(stats)
        
    except Exception as e:
        print(f"❌ Lỗi trong quá trình thực hiện: {e}")
    
    finally:
        conn.close()
        print("\n✅ Đã đóng kết nối database")

if __name__ == "__main__":
    main()
